In [ ]:
!pip install langchain openai faiss-cpu chromadb flask flask-restful beautifulsoup4 ollama

!pip install -U langchain-community

!pip install sentence-transformers

You should consider upgrading via the 'C:\Users\91832\Desktop\Task_llm\task\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\91832\Desktop\Task_llm\task\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from langchain.document_loaders import WebBaseLoader


url = "https://brainlox.com/courses/category/technical"
loader = WebBaseLoader(url)
documents = loader.load()
html_content = documents[0].page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
print(html_content[:1000])  # Show the first 3 documents if available


Brainlox: Learn technical courses.Courses TechnicalAcademicLanguageMusicLifestyleBook a Free Demo NowSign InFAQContact UsHomeCoursesCoursesWe found great courses available for you$30per sessionLEARN SCRATCH PROGRAMING
Scratch Course is the foundation of coding and is a building block of a coding journey. If you want 16 LessonsView Details$30per sessionLEARN CLOUD COMPUTING BASICS-AWS
In this course we are going to cover the basics and the most important services on AWS,
At the end  20 LessonsView Details$30per sessionLEARN MOBILE DEVELOPMENT
Mobile application development is the process of creating software applications that run on a mobil 24 LessonsView Details$30per sessionLEARN CORE JAVA PROGRAMMING ONLINE
Java is a very popular high-level, class-based, object-oriented programming language that is design 41 LessonsView Details$30per sessionLEARN ROBOTICS
You can open all kinds of doors for advancement in so many careers with a basic understanding of el 25 LessonsView Details$30per s

In [15]:
!pip install nltk

You should consider upgrading via the 'C:\Users\91832\Desktop\Task_llm\task\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import re

# Improved Regex pattern to extract course information
pattern = re.compile(
    r'\$(\d+)\s*per session\s*(.*?)\s*(\d+)\s*Lessons\s*View Details',
    re.DOTALL
)

# Find all matches in the scraped data
matches = pattern.findall(html_content)

# Structure the data into a list of dictionaries
courses = []
for match in matches:
    price = f"${match[0]} per session"
    content = match[1].strip()  # Full content (title + description)
    lessons = f"{match[2]} Lessons"

    # Step 1: Split content into title and description
    # Look for natural breaks (e.g., colons, newlines, or sentence boundaries)
    if ":" in content:
        # Split on the first colon
        title, description = content.split(":", 1)
    elif "\n" in content:
        # Split on the first newline
        title, description = content.split("\n", 1)
    else:
        # Fallback to length-based splitting
        words = content.split()
        if len(words) > 5:  # Assume the first 5 words are the title
            title = " ".join(words[:5])
            description = " ".join(words[5:])
        else:
            title = content
            description = ""

    # Step 2: Clean up the title and description
    title = title.strip()
    description = description.strip()

    # Remove any leading/trailing colons or spaces
    if description.startswith(":"):
        description = description[1:].strip()

    course = {
        "price": price,
        "title": title,
        "description": description,
        "lessons": lessons,
        "link": "#"  # Placeholder for link
    }
    courses.append(course)

# Print the structured data
for course in courses:
    print(course)

{'price': '$30 per session', 'title': 'LEARN SCRATCH PROGRAMING', 'description': 'Scratch Course is the foundation of coding and is a building block of a coding journey. If you want', 'lessons': '16 Lessons', 'link': '#'}
{'price': '$30 per session', 'title': 'LEARN CLOUD COMPUTING BASICS-AWS', 'description': 'In this course we are going to cover the basics and the most important services on AWS,\nAt the end', 'lessons': '20 Lessons', 'link': '#'}
{'price': '$30 per session', 'title': 'LEARN MOBILE DEVELOPMENT', 'description': 'Mobile application development is the process of creating software applications that run on a mobil', 'lessons': '24 Lessons', 'link': '#'}
{'price': '$30 per session', 'title': 'LEARN CORE JAVA PROGRAMMING ONLINE', 'description': 'Java is a very popular high-level, class-based, object-oriented programming language that is design', 'lessons': '41 Lessons', 'link': '#'}
{'price': '$30 per session', 'title': 'LEARN ROBOTICS', 'description': 'You can open all kinds

In [4]:
# Combine relevant fields into a single text string (excluding 'link')
course_texts = [
    f"Title: {course['title']}\nDescription: {course['description']}\nPrice: {course['price']}\nLessons: {course['lessons']}"
    for course in courses
]

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Size of each chunk (in characters)
    chunk_overlap=50,  # Overlap between chunks (in characters)
)

# Split the course texts into chunks
text_chunks = []
for text in course_texts:
    chunks = text_splitter.split_text(text)
    text_chunks.extend(chunks)

In [6]:
# Print the chunks
for i, chunk in enumerate(text_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n{'-'*50}\n")

Chunk 1:
Title: LEARN SCRATCH PROGRAMING
Description: Scratch Course is the foundation of coding and is a building block of a coding journey. If you want
Price: $30 per session
Lessons: 16 Lessons
--------------------------------------------------

Chunk 2:
Title: LEARN CLOUD COMPUTING BASICS-AWS
Description: In this course we are going to cover the basics and the most important services on AWS,
At the end
Price: $30 per session
Lessons: 20 Lessons
--------------------------------------------------

Chunk 3:
Title: LEARN MOBILE DEVELOPMENT
Description: Mobile application development is the process of creating software applications that run on a mobil
Price: $30 per session
Lessons: 24 Lessons
--------------------------------------------------

Chunk 4:
Title: LEARN CORE JAVA PROGRAMMING ONLINE
Description: Java is a very popular high-level, class-based, object-oriented programming language that is design
Price: $30 per session
Lessons: 41 Lessons
---------------------------------------

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5396\2837024061.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [9]:
from langchain.vectorstores import FAISS

# Create the FAISS vector store
vector_store = FAISS.from_texts(text_chunks, embeddings)

In [10]:
vector_store.save_local("faiss_index")


In [11]:
vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


In [12]:

# Increase the number of retrieved documents
retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Retrieve top 5 documents


In [27]:
from flask import Flask, request, jsonify
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize Flask app
app = Flask(__name__)

# Load the embeddings model (same as used in your .py file)
try:
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    print("Embeddings model loaded successfully!")
except Exception as e:
    print(f"Error loading embeddings model: {e}")

# Load the FAISS vector store
try:
    vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    print("FAISS vector store loaded successfully!")
except Exception as e:
    print(f"Error loading FAISS vector store: {e}")

# Create a retriever for similarity search
retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Retrieve top 3 documents

# Root endpoint for testing
@app.route('/')
def home():
    return "Flask app is running! Use the /chat endpoint to interact with the chatbot."

# Define the /chat endpoint
@app.route('/chat', methods=['POST'])
def chat():
    try:
        # Get the user query from the request
        data = request.get_json()
        print("Received request data:", data)  # Debugging line
        query = data.get("query")

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Perform a similarity search using the retriever
        print("Performing similarity search for query:", query)  # Debugging line
        relevant_docs = retriever.get_relevant_documents(query)
        print("Relevant documents found:", relevant_docs)  # Debugging line

        # Format the results
        results = []
        for doc in relevant_docs:
            result = {
                "content": doc.page_content,  # The text content of the relevant document
            }
            results.append(result)

        # Return the results as JSON
        return jsonify({"results": results}), 200

    except Exception as e:
        print(f"Error in /chat endpoint: {e}")  # Debugging line
        return jsonify({"error": str(e)}), 500

# Run the Flask app in Jupyter Notebook
if __name__ == '__main__':
    import threading
    threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

Embeddings model loaded successfully!
FAISS vector store loaded successfully!
 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [28]:
import requests

# Send a POST request to the /chat endpoint
response = requests.post(
    "http://127.0.0.1:5000/chat",
    json={"query": "Find Python courses"}
)

# Print the response
print(response.json())

{'results': [{'content': 'Title: PYTHON PROGRAMMING GROUP CLASSES - BEGINNER\nDescription: Python is a language with simple syntax, and a powerful set of libraries. It has a rich programming\nPrice: $30 per session\nLessons: 16 Lessons'}, {'content': 'Title: PYTHON PROGRAMMING-ADVANCEIf you already know\nDescription: Python basics, then this training is the next step in your Python learning path\nPrice: $35 per session\nLessons: 30 Lessons'}, {'content': 'Title: PYTHON PROGRAMMING-INTERMEDIATE\nDescription: Take your python skills to the next level and start building real applications.\nPython is a pro\nPrice: $32 per session\nLessons: 16 Lessons'}]}
